In [8]:
import pandas
import json


# modelop.init
def init() -> None:
    """A function to load cost parameters and field names as global vars."""

    # Load cost constants and field names as global variables
    global COST_OF_ONE_SURPLUS, COST_OF_ONE_DEFICIT, COST_OF_ONE_MODEL_INFERENCE, COST_OF_ONE_HUMAN_INFERENCE
    global ACTUALS_FIELD, MODEL_PRED_FIELD, HUMAN_PRED_FIELD

    model_parameters = json.load(open("./model_parameters.json", "r"))
    COST_OF_ONE_SURPLUS = model_parameters.get("COST_OF_ONE_SURPLUS")
    COST_OF_ONE_DEFICIT = model_parameters.get("COST_OF_ONE_DEFICIT")
    COST_OF_ONE_MODEL_INFERENCE = model_parameters.get("COST_OF_ONE_MODEL_INFERENCE")
    COST_OF_ONE_HUMAN_INFERENCE = model_parameters.get("COST_OF_ONE_HUMAN_INFERENCE")

    ACTUALS_FIELD = model_parameters.get("ACTUALS_FIELD")
    MODEL_PRED_FIELD = model_parameters.get("MODEL_PRED_FIELD")
    HUMAN_PRED_FIELD = model_parameters.get("HUMAN_PRED_FIELD")


# modelop.metrics
def metrics(data: pandas.DataFrame) -> dict:
    """A function to compute ROI metrics on input data

    Args:
        data (pandas.DataFrame): Input data containing model and human forecasts,
            as well as actuals.

    Returns:
        dict: Cost-performance metrics and overall ROI.
    """

    # Calculate over/under predictions by model and humans
    data["model_surplus"] = (data[MODEL_PRED_FIELD] - data[ACTUALS_FIELD]).clip(0)
    data["model_deficit"] = (data[ACTUALS_FIELD] - data[MODEL_PRED_FIELD]).clip(0)

    data["human_surplus"] = (data[HUMAN_PRED_FIELD] - data[ACTUALS_FIELD]).clip(0)
    data["human_deficit"] = (data[ACTUALS_FIELD] - data[HUMAN_PRED_FIELD]).clip(0)

    # Assign a dollar amount to each forecast
    data[["model_surplus", "human_surplus"]] *= COST_OF_ONE_SURPLUS
    data[["model_deficit", "human_deficit"]] *= COST_OF_ONE_DEFICIT

    # Compute totals
    sums = data.sum(axis=0)

    model_surplus_total = sums["model_surplus"]
    model_deficit_total = sums["model_deficit"]
    human_surplus_total = sums["human_surplus"]
    human_deficit_total = sums["human_deficit"]

    # Add Operation Cost
    model_total_cost = model_surplus_total + model_deficit_total + COST_OF_ONE_MODEL_INFERENCE*data.shape[0]
    human_total_cost = human_surplus_total + human_deficit_total + COST_OF_ONE_HUMAN_INFERENCE*data.shape[0]

    return {
        # Top-level key metrics
        "is_model_more_effective": bool(human_total_cost - model_total_cost >= 0),
        "cost_savings_by_model": human_total_cost - model_total_cost,
        # More in-depth view
        "business_value": [
            {
                "test_name": "Actual ROI",
                "test_category": "business_value",
                "test_type": "actual_roi",
                "test_id": "business_value_actual_roi",
                "values": {
                    "model_surplus_total": model_surplus_total,
                    "model_deficit_total": model_deficit_total,
                    "human_surplus_total": human_surplus_total,
                    "human_deficit_total": human_deficit_total,
                    "model_total_cost": model_total_cost,
                    "human_total_cost": human_total_cost,
                    "is_model_more_effective": bool(
                        human_total_cost - model_total_cost >= 0
                    ),
                    "cost_savings_by_model": human_total_cost - model_total_cost,
                },
            }
        ],
    }


In [9]:
# Load params
init()

In [10]:
# read sample data
data = pandas.read_json("sample_data.json", lines=True)

In [11]:
data.head()

,forecast_ID,forecast_date,actual_deliveries,predicted_deliveries,human_forecast
0,AUG012022,08/01/2022,42,42,42
1,AUG022022,08/02/2022,43,44,45
2,AUG032022,08/03/2022,44,43,42
3,AUG042022,08/04/2022,45,47,49
4,AUG052022,08/05/2022,46,44,42


In [12]:
results = metrics(data)

In [13]:
results

{'is_model_more_effective': True,
 'cost_savings_by_model': 22590,
 'business_value': [{'test_name': 'Actual ROI',
   'test_category': 'business_value',
   'test_type': 'actual_roi',
   'test_id': 'business_value_actual_roi',
   'values': {'model_surplus_total': 9000,
    'model_deficit_total': 13500,
    'human_surplus_total': 18000,
    'human_deficit_total': 27000,
    'model_total_cost': 22510,
    'human_total_cost': 45100,
    'is_model_more_effective': True,
    'cost_savings_by_model': 22590}}]}

In [7]:
pandas.DataFrame([results["business_value"][0]["values"]])

,model_surplus_total,model_deficit_total,human_surplus_total,human_deficit_total,model_total_cost,human_total_cost,is_model_more_effective,cost_savings_by_model
0,9100,13600,19000,28000,22700,47000,True,24300
